In [1]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [4]:
import os

In [5]:
file_path=r"Global_Cybersecurity_Threats_2015-2024.csv"

In [6]:
if os.path.exists(file_path):
    try:
        df = pd.read_csv(file_path)
        print("File loaded successfully!")
        print(df.head())
    except FileNotFoundError:
        print("Error: File not found.")

    except PermissionError:
        print("Error: You don’t have permission to access this file.")

    except pd.errors.EmptyDataError:
        print("Error: The file is empty.")

    except pd.errors.ParserError:
        print("Error: Could not parse the file (might be corrupted or not CSV).")

    except ValueError as ve:
        print("ValueError:", ve)

    except Exception as e:
        print("An unexpected error occurred:", e)

else:
    print("Error: File path does not exist.")   

File loaded successfully!
   Country  Year        Attack Type     Target Industry  \
0    China  2019           Phishing           Education   
1    China  2019         Ransomware              Retail   
2    India  2017  Man-in-the-Middle                  IT   
3       UK  2024         Ransomware  Telecommunications   
4  Germany  2018  Man-in-the-Middle                  IT   

   Financial Loss (in Million $)  Number of Affected Users Attack Source  \
0                          80.53                    773169  Hacker Group   
1                          62.19                    295961  Hacker Group   
2                          38.65                    605895  Hacker Group   
3                          41.44                    659320  Nation-state   
4                          74.41                    810682       Insider   

  Security Vulnerability Type Defense Mechanism Used  \
0          Unpatched Software                    VPN   
1          Unpatched Software               Firewa

In [7]:
df=df.drop_duplicates()

In [8]:
df=df.dropna()

In [9]:
df=df.fillna("Not Mentioned")

In [10]:
df["Country"]=df["Country"].str.title()

In [11]:
df["Attack Type"]=df["Attack Type"].str.lower().str.strip()
df["Target Industry"]=df["Target Industry"].str.title()
df["Attack Source"]=df["Attack Source"].str.title()
df["Defense Mechanism Used"]=df["Defense Mechanism Used"].str.title()

In [12]:
df=df.rename(columns={
    "Financial Loss (in Million $)":"Financial_Loss",
    "Number of Affected Users":"Affected_Users",
    "Incident Resolution Time (in Hours)":"Resolution_Hours",
    "Attack Type":"Attack_Type",
    "Target Industry":"Target_Industry",
    "Attack Source":"Attack_Source",
    "Security Vulnerability Type":"Vulnerability_Type",
    "Defense Mechanism Used":"Defense Mechanism"
})

In [13]:
print(df.head())
print(df.info())

   Country  Year        Attack_Type     Target_Industry  Financial_Loss  \
0    China  2019           phishing           Education           80.53   
1    China  2019         ransomware              Retail           62.19   
2    India  2017  man-in-the-middle                  It           38.65   
3       Uk  2024         ransomware  Telecommunications           41.44   
4  Germany  2018  man-in-the-middle                  It           74.41   

   Affected_Users Attack_Source  Vulnerability_Type   Defense Mechanism  \
0          773169  Hacker Group  Unpatched Software                 Vpn   
1          295961  Hacker Group  Unpatched Software            Firewall   
2          605895  Hacker Group      Weak Passwords                 Vpn   
3          659320  Nation-State  Social Engineering  Ai-Based Detection   
4          810682       Insider  Social Engineering                 Vpn   

   Resolution_Hours  
0                63  
1                71  
2                20  
3         

In [14]:
pip install mysql-connector-python SQLAlchemy

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install mysql-connector-python SQLAlchemy

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install sql alchemy

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [18]:
!bash -c "pip install mysql-connector-python"

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [20]:
import mysql.connector
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="siri@123"
)
cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS Cyber_threats")
print("Database 'Cyber_threats' created successfully!")
cursor.close()
conn.close()

InterfaceError: 2002: Can't connect to local MySQL server through socket '%-.100s' (%s) (Warning: not all arguments converted during string formatting)